In [1]:
import os, json, time
from gridappsd import GridAPPSD, topics as t
from gridappsd.topics import service_input_topic, service_output_topic
from distributedtopology import DistributedTopology
from topologydictionary import TopologyDictionary
from networkmodel import NetworkModel

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

# Connect to GridAPPS-D Platform
gapps = GridAPPSD()
assert gapps.connected



In [2]:
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA"

#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F"  # 13 assets
#model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus

In [3]:
Topology = TopologyDictionary(gapps, model_mrid)


In [4]:
network = NetworkModel(gapps)


In [5]:
network.build_equip_dicts(model_mrid, Topology)

In [6]:
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine', 'PowerElectronicsConnection']
Topology.build_linknet(EqTypes)


i2 =  3908
old_counter =  0
i2 =  10
old_counter =  7818
i2 =  1292
old_counter =  7840
i2 =  8
old_counter =  10426
i2 =  179
old_counter =  10435


In [9]:
Topology.update_switches()


In [10]:
Topology.build_feeder_islands()

In [ ]:
i1 = 0
root_type = list(Topology.Feeders['feeder_'+ str(i1)].keys())[1]
root_id = Topology.Feeders['feeder_'+ str(i1)][root_type]
root_node = Topology.Feeders['feeder_'+ str(i1)]['root_node']


In [20]:
Topology.ConnNodeDict['_E0D6F386-AFB8-4037-BEC9-96963A9E866C']

{'name': 'm1026chp-2',
 'TopologicalNode': '_F0E66C9C-AA33-4715-8876-8ADD7F0D0E78',
 'nominalVoltage': '12470',
 'ACLineSegment': ['_8BE12AAB-5519-4683-AB41-58B556CC9F3F'],
 'Breaker': [],
 'EnergyConsumer': [],
 'Fuse': [],
 'House': [],
 'LinearShuntCompensator': [],
 'LoadBreakSwitch': ['_60B9C35A-A7B1-4295-AA5B-7E4E21EEB082'],
 'PowerTransformer': [],
 'RatioTapChanger': [],
 'Recloser': [],
 'TransformerTank': [],
 'SynchronousMachine': [],
 'PowerElectronicsConnection': [],
 'Measurement': ['_16e36a36-3b58-4568-a7b4-d2a7b4f4d866',
  '_2c3d8060-89b9-4840-ba01-03fb34e5213f',
  '_4a33262d-cf92-41df-8655-351a16f4d064',
  '_14ade66a-c753-448e-beaa-0376d63c762a',
  '_733cc140-a5ef-4df3-a453-813eb8a1a580',
  '_f4e5e2dc-5615-4534-8d1e-3ad12d40e225'],
 'Feeder': [],
 'Island': [],
 'node': 3021,
 'list': 3875}

In [ ]:
Topology.EquipDict[root_type][root_id]

In [ ]:
Topology.EquipDict['PowerElectronicsConnection']

In [24]:
Topology.ConnNodeDict[Topology.NodeList[3021]]

{'name': 'm1026chp-3',
 'TopologicalNode': '_FE3AD568-781F-4EC9-AA13-4571E90DE832',
 'nominalVoltage': '12470',
 'ACLineSegment': ['_8BE12AAB-5519-4683-AB41-58B556CC9F3F'],
 'Breaker': [],
 'EnergyConsumer': [],
 'Fuse': [],
 'House': [],
 'LinearShuntCompensator': [],
 'LoadBreakSwitch': [],
 'PowerTransformer': [],
 'RatioTapChanger': [],
 'Recloser': [],
 'TransformerTank': [],
 'SynchronousMachine': ['_0F866CD6-FB46-440F-A828-B6973E55E4D7'],
 'PowerElectronicsConnection': [],
 'Measurement': ['_1c431dc4-b08b-4f2d-ab39-1d2d1e5f56a6',
  '_3a413f2f-de4a-42b4-8731-88030bc95cd3',
  '_9dbd2e9a-4ba1-46d6-9d10-5db09eeb0797',
  '_910e8054-754c-473f-8efd-35823e720b0e',
  '_98577337-6d82-4e90-a72b-9d52b34e23c8',
  '_c667d756-6cf8-4c3e-b8ac-794f2301f832',
  '_3125010b-bdb7-4a45-83cd-4c03df356037',
  '_64f44c12-71d1-465c-8ddc-1ad21a0a9639',
  '_7afceffb-89f1-4cf7-b4dc-aad9d7a4d32b'],
 'Feeder': ['feeder_2'],
 'Island': [],
 'node': 3022,
 'list': 0}

In [7]:
Topology.TerminalsDict[Topology.TermList[10434]]

{'ConnectivityNode': '_EEB13278-5AE5-4401-BBF6-5F3A1CC48080',
 'term': 10435,
 'next': 0,
 'far': 5290}

In [22]:
Topology.TerminalsDict[Topology.TermList[3874]]

{'ConnectivityNode': '_E0D6F386-AFB8-4037-BEC9-96963A9E866C',
 'term': 3875,
 'next': 2768,
 'far': 3022}

In [26]:
Topology.ConnNodeDict[Topology.TerminalsDict[Topology.TermList[2767]]['ConnectivityNode']]

{'name': 'm1026chp-1',
 'TopologicalNode': '_F0E66C9C-AA33-4715-8876-8ADD7F0D0E78',
 'nominalVoltage': '12470',
 'ACLineSegment': ['_FC494784-7476-45C1-AD8A-3248C54D25E6'],
 'Breaker': [],
 'EnergyConsumer': [],
 'Fuse': [],
 'House': [],
 'LinearShuntCompensator': [],
 'LoadBreakSwitch': ['_60B9C35A-A7B1-4295-AA5B-7E4E21EEB082'],
 'PowerTransformer': [],
 'RatioTapChanger': [],
 'Recloser': [],
 'TransformerTank': [],
 'SynchronousMachine': [],
 'PowerElectronicsConnection': [],
 'Measurement': ['_5354c11e-dcc0-46eb-bc7d-6b46d0d34156',
  '_63774677-8878-45b8-9746-103da968d098',
  '_a8631936-c785-46ce-9d93-5100e0fa9d1b',
  '_0725a1f8-0984-4a11-986a-be64758f9c72',
  '_727a90d2-abfd-46b7-b59a-c830044f5e2b',
  '_cd834ef3-b9cc-4b36-a399-37604bfdc3ce',
  '_492ef477-2b49-4149-99a4-654d99453edd',
  '_795870d2-8413-4883-a8ec-08aed90637ac',
  '_d1129a1e-1e39-44cd-b000-4c675642e3a9'],
 'Feeder': ['feeder_2'],
 'Island': [],
 'node': 2219,
 'list': 3875}

In [16]:
Topology.EquipDict['PowerElectronicsConnection']['_DAD1B6F0-D1E6-43BF-8E49-485DDA86EE31']

{'name': 'pv_1097',
 'node1': '_004BA4B9-78D5-45CF-AFDA-22EA40CEC73B',
 'term1': '_ACF10D38-B131-4D8B-BFD0-9967AAC52D27'}

In [ ]:
Topology.TerminalsDict[Topology.TermList[10603]]